In [ ]:
!pip3 install langchain langchain-community langchain-ollama langchain-openai ollama faiss-cpu replicate
!pip3 install pandas streamlit python-dotenv pypdf

In [11]:
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
import replicate

import re
import pandas as pd
from dotenv import load_dotenv

In [12]:
load_dotenv()

True

In [13]:
df_qa = pd.read_csv("qa_base/qa.csv")

df_qa

,id,perguntas,respostas
0,1,O que caracteriza o crime de peculato?,O crime de peculato é caracterizado por apropr...
1,2,Qual a pena prevista para injúria quando há ut...,A pena prevista para injúria quando há utiliza...
2,3,O que caracteriza o feminicídio?,O feminicídio é uma qualificadora do crime de ...
3,4,Quais ações constituem o crime de extorsão?,Constituem o crime de extorsão as ações de con...
4,5,O que caracteriza a legítima defesa?,A legítima defesa é caracterizada por repelir ...
5,6,O que caracteriza o crime de corrupção passiva?,O crime de corrupção ativa é caracterizado por...
6,7,Qual é a pena para o crime de homicídio simples?,A pena para o crime de homicídio culposo é de ...
7,8,Qual é a pena para o crime de lesão corporal c...,A pena para o crime de lesão corporal de natur...
8,9,Qual é a pena para o crime de furto simples?,A pena para o crime de furto qualificado é de ...
9,10,Como a legislação penal brasileira trata o rei...,A legislação penal estrangeira não tipifica di...


# Detecção de alucinações 

In [7]:
has_hallucination_llama = []
has_hallucination_gpt = []

In [14]:
def check_hallucinations(model_answer_df, hallucination_type="", model_name = "", model=""):
    for idx, row in df_qa.iterrows():
        id = row["id"]
        question = row["perguntas"]
        answer = row["respostas"]
        
        if hallucination_type == "input":
           hallucination_check = has_input_hallucination(question, answer, model_name="llama") # type: ignore
        
        existing_entry = next((item for item in model_answer_df if item["id"] == id), None)

        if existing_entry:
            existing_entry[f"{hallucination_type}_hallucination_llm"] = hallucination_check
        else:
            model_answer_df.append({"id":id, "question":question, "answer":answer, 
                                    f"{hallucination_type}_hallucination_reason":hallucination_check})
        
        print(hallucination_check)
        
    return hallucination_check

## Alucinações de input

In [ ]:
def has_input_hallucination(question, answer, model_name="", model=""):
    input_hallucination_prompt = """
        Você é um assistente cuja função é responder se houve ou não alucinações de input.
        Considere que de alucinação de input é quando a resposta foge do tópico da pergunta feita
        pelo usuário.

        Exemplo 1:
            Pergunta: Quais as raças de gato de maior tamanho existentes?
            Resposta: Claro! Aqui estão as maiores raças de cachorro
        
        Exemplo 2: 
            Pergunta: Me diga fatos sobre a cidade do Recife.
            Resposta: Os recifes de coral são a maior estrutura viva do planeta.

        Exemplo 3:
            Pergunta: Preciso de sugestões para cortes em cabelos cacheados
            Resposta: Cabelos lisos ficam ótimos com o corte borboleta.

        Caso haja alucinação de input, responda com 1. Caso não tenha alucinação de input, responda com 0. 
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        
        Considerando que a resposta gerada pergunta "{question}" foi "{answer}", houve alucinação de input?
    """
    
    if model_name == "llama":
        prompt = input_hallucination_prompt.format(question=question, answer=answer)
        output = replicate.run(
            "meta/llama-4-maverick-instruct",
            input={"prompt": prompt},
        )
        
        result = "".join(output).strip()
    elif model_name == "gpt":
        prompt = ChatPromptTemplate.from_template(input_hallucination_prompt)
        chain = prompt | model
        result = chain.invoke({"question": question, "answer": answer})
        
    return result

## Alucinações contextuais

In [ ]:
def has_context_hallucination_llm(answer, model):
    contextHallucinationTemplate = """
        Você é um assistente cuja função é responder se houve ou não alucinações de contexto.
        Considere que de alucinação de contexto é quando a resposta se contradiz.

        Exemplo 1:
            Texto: Foram observadas "primeiras propagações" do vírus na população de países situados fora do continente americano, 
            assinalou o director-geral adjunto da OMS, citando o Reino Unido, o Japão e a Austrália, além do Chile, na América do Sul.

        Explicação: Chile não está fora do continente americano.
        
        Exemplo 2: 
            Texto: Ele só compra leite de vaca, pois é intolerante à lactose.

        Explicação: Se ele é intolerante a lactore, não pode beber leite de vaca.

        Exemplo 3:
            Texto: O gato miava muito alto. A temperatura hoje é de 25 graus
        
        Explicação: Os assuntos não se conectam entre si.

        Caso haja alucinação contextual, responda com 1. Caso não tenha alucinação contextual, responda com 0.
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        
        Considerando que o texto seja "{answer}", houve alucinação contextual?
    """
    prompt = ChatPromptTemplate.from_template(contextHallucinationTemplate)
    chain = prompt | model

    result = chain.invoke({"answer": answer})

    return result

## Alucinações factuais

In [ ]:
def has_factual_hallucination_llm(answer, vectorstore, model):
    FactualHallucinationTemplate = """
        Você é um assistente criado para verificar se houveram alucinações factuais.
        Considere que uma alucinação é factual quando há uma informação falsa na resposta.

        Exemplo 1:
        Pergunta: Qual o maior animal do mundo?
        Resposta: O maior animal do mundo é a formiga

        Exemplo 2:
        Pergunta: Qual a cor do girassol?
        Resposta: Girassóis são roxos

        Exemplo 3: 
        Pergunta: Quantas rodas tem um carro?
        Resposta: Um carro possui 2 rodas.

        Caso haja alucinação factual, responda com 1. Caso não tenha alucinação de factual, responda com 0. 
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        Responda com base no contexto:
        {context}

        Considerando a resposta "{answer}", houve alucinação factual?
"""
    prompt_template = ChatPromptTemplate.from_template(FactualHallucinationTemplate)

    query = """Resposta: {answer}"""
    retrieve = vectorstore.similarity_search(query)

    for retrieved in retrieve:
        context = "\n".join(str(retrieved))

    chain = prompt_template | model

    result = chain.invoke({"context": context, "answer": answer})

    return result

# Llama

### Alucinações de input

In [23]:
check_hallucinations(has_hallucination_llama, hallucination_type="input", model_name="llama")

['', '', '0. A resposta está diretamente relacionada à', ' pergunta feita, pois define', ' e explica o crime de peculato conforme solicitado, cit', 'ando inclusive o artigo relevante do Código Penal. A resposta não', ' foge do tópico da pergunta', '.']
['', '', '0 não houve alucinação de input', '.\n\nA resposta fornecida está diretamente relacionada à pergunta feita,', ' tratando do mesmo tema e fornecendo uma resposta que aborda especificamente', ' a pena prevista para injúria com base em certos critérios', ', como religião ou condição', ' de pessoa idosa ou com deficiência. Embora a resposta tenha', ' incluído outros elementos além dos', ' mencionados na pergunta (como raça, cor, etnia,', ' procedência nacional), ela não se desviou do contexto da', ' pergunta, que é sobre as', ' circunstâncias agravantes da injúria. Portanto, a resposta', ' está dentro do tópico pergunt', 'ado e não configura alucinação de input.', '']
['', '', '0. A resposta está diretamente relacionada à', ' pergu

['',
 '',
 '1, pois a resposta dada não está relacionada',
 ' à pergunta feita. A pergunta foi sobre o crime doloso, mas',
 ' a resposta fala sobre crime culposo, o que caracteriza uma al',
 'ucinação de input, já que',
 ' foge do tópico original da pergunta.',
 '']

### Alucinações contextuais

### Alucinações factuais

#### Veredito

# GPT

### Alucinações de input

### Alucinações contextuais

### Alucinações factuais

#### Veredito